# Set up

Load libraries

In [1]:
import os
import sys, getopt
from pathlib import Path
import xml.etree.cElementTree as ET
import pandas as pd

Libraries for database access

In [2]:
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs

Here some instructions [on how to connect to gsheets API](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c).

In [6]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os
import pickle
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


# Read data from different sources

## Read invetory from Google Spreadsheet

In [7]:
def read_gsheet(SPREADSHEET_ID,RANGE_NAME, JSONFILE):
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(JSONFILE, SCOPES) 
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

In [8]:
# 
conf_json = Path(os.path.expanduser('~')) / '.secrets/googlecloud/python-gsheet-api.json'
Lilas_doc='1zwNuc82Opu3Kxd93YRzbzUXjP3RZzJyR' # this does not work, maybe due to format (XLSX instead of gsheets?)
my_doc='105V_zTZ-Hzj1ZuZDt1HFo750viegjmM2zyfPvJmPDF4'
read_gsheet(my_doc,'Assessment Target!A1:AA10000',conf_json)

In [9]:
values_input[10]

['Strategic',
 'English_SWA_2015',
 'English_SWA_2015_1',
 'Shrublands on southern Swan Coastal Plain Ironstone',
 '2015',
 'English',
 'IUCN RLE 2.0',
 'Sub-global/Sub-national',
 'Terrestrial',
 'T3. Shrublands and shrubby woodlands',
 'T3.2 Seasonally dry temperate heath and shrublands',
 'DB Public',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'CR',
 '',
 'Oceania',
 'AU',
 '',
 '',
 'too expensive']

In [17]:

inventory=pd.DataFrame(values_input[1:], columns=values_input[0])

In [18]:
#inventory
inventory['Assessment ID']=inventory['Ecosystem ID'].str.replace('_[0-9]+$', '', regex=True)

In [19]:
# inventory.columns
inventory[inventory['Ecosystem ID']=='Benson_VineThicket_2013_1'][['Reference label', 'Assessment ID', 'Ecosystem ID', 'AT-Name',
       'Realm\nGET_Level 1', 'Biome\nGET_Level 2',
       'Functional Group\nGET_Level 3']]

,Reference label,Assessment ID,Ecosystem ID,AT-Name,Realm\nGET_Level 1,Biome\nGET_Level 2,Functional Group\nGET_Level 3
2,Keith_Foundations_2013,Benson_VineThicket_2013,Benson_VineThicket_2013_1,Mock Olive - Wilga - Peach Bush - Carissa dry ...,Terrestrial,T1. Tropical-subtropical forests,T1.2 Tropical-subtropical dry forests and scrubs


In [20]:
selected_columns = ['Reference label', 'Assessment ID', 'Ecosystem ID', 'AT-Name',
       'Realm\nGET_Level 1', 'Biome\nGET_Level 2',
       'Functional Group\nGET_Level 3']
inv_df = inventory[selected_columns]

## Read data from GET database

In [175]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'iucnecoadm'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

In [194]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)

In [195]:
qry = """
SELECT eco_id,eco_name,eco_name_orig,external_code,efg_code,f.name,level,membership,assigned_by 
FROM rle.assessment_get_xwalk
LEFT JOIN rle.assessment_units USING(eco_id)
LEFT JOIN functional_groups f ON efg_code=f.code
"""
cur.execute(qry)
xwalks = cur.fetchall()
xwalk_df=pd.DataFrame(xwalks,
                columns=['eco_id','eco_name','eco_name_orig','external_code','efg_code','efg_name','level','membership','assigned_by'])

In [196]:
qry = """
SELECT asm_id,eco_id,eco_name,eco_name_orig, eco_name_lang,external_code,assessment_date,
overall_risk_category,risk_category_bounds
FROM rle.assessment_units u
JOIN rle.assessment_overall USING(eco_id)
"""
cur.execute(qry)
assm_units = cur.fetchall()
assm_units_df=pd.DataFrame(assm_units,
                columns=['asm_id','eco_id','eco_name','eco_name_orig', 'eco_name_lang','external_code','assessment_date',
'overall_risk_category','risk_category_bounds'])

In [197]:
assm_units_df

,asm_id,eco_id,eco_name,eco_name_orig,eco_name_lang,external_code,assessment_date,overall_risk_category,risk_category_bounds
0,Carre_RLE_France_2012,Carre_RLE_France_2012:1,Le bassin d’Arcachon,Le bassin d’Arcachon,en,Carre_RLE_France_2012:1,2012-01-01,None,None
1,Carre_RLE_France_2012,Carre_RLE_France_2012:2,Les lacs et étangs du littoral aquitain,Les lacs et étangs du littoral aquitain,en,Carre_RLE_France_2012:2,2012-01-01,None,None
2,Carre_RLE_France_2012,Carre_RLE_France_2012:3,Les lagunes méditerranéennes françaises,Les lagunes méditerranéennes françaises,en,Carre_RLE_France_2012:3,2012-01-01,None,None
3,Carre_RLE_France_2012,Carre_RLE_France_2012:4,Les marais maritimes atlantiques,Les marais maritimes atlantiques,en,Carre_RLE_France_2012:4,2012-01-01,None,None
4,Holdaway_Uncommon_NewZealand_2012,Holdaway_Uncommon_NewZealand_2012:1,Active sand dune,Active sand dune,en,Holdaway_Uncommon_NewZealand_2012:1,2012-01-01,None,None
...,...,...,...,...,...,...,...,...,...
2268,Javed_RLE_AbuDhabi_2021,Javed_RLE_AbuDhabi_2021_12,Saltmarshes,Saltmarshes,sp,Javed_RLE_AbuDhabi_2021_12,2021-01-01,EN,None
2269,Javed_RLE_AbuDhabi_2021,Javed_RLE_AbuDhabi_2021_13,Seagrass,Seagrass,sp,Javed_RLE_AbuDhabi_2021_13,2021-01-01,VU,None
2270,Javed_RLE_AbuDhabi_2021,Javed_RLE_AbuDhabi_2021_14,Coral reefs,Coral reefs,sp,Javed_RLE_AbuDhabi_2021_14,2021-01-01,EN,None
2271,Javed_RLE_AbuDhabi_2021,Javed_RLE_AbuDhabi_2021_15,Tidal flats,Tidal flats,sp,Javed_RLE_AbuDhabi_2021_15,2021-01-01,VU,None


In [198]:
qry = """
SELECT asm_id,ref_code
FROM rle.assessments
"""
cur.execute(qry)
assms = cur.fetchall()
assms_df=pd.DataFrame(assms,
                columns=['asm_id','ref_code'])

In [202]:
assms_df

,asm_id,ref_code
0,Deltas_at_Risk,Tessler et al. 2015
1,Bland_MarineEcosystem_2018,Bland et al. 2018
2,Comer_ForestsAmerica_2020,Comer et al. 2020
3,Gubbay_EuropeHabitat_Marine_2016,Gubbay_Europe_2017
4,Etter_RLE_Colombia_2015,Etter et al. 2015
...,...,...
96,Poulin_EuropeanReedbeds_2013,Keith et al. 2013
97,Pliscoff_RLE_Chile_2015,Pliscoff 2015
98,Tan_SouthWestChina_2017,Tan et al. 2017
99,Wardle_GeorginaGidgee_2015,Wardle et al. 2015


In [200]:
assms_df.ref_code.values

array(['Tessler et al. 2015', 'Bland et al. 2018', 'Comer et al. 2020',
       'Gubbay_Europe_2017', 'Etter et al. 2015', 'IUCN_Brazil_RLE_2015',
       'Keith et al. 2013', 'Ghoraba et al. 2021', 'Huss et al. 2017',
       'IUCN_FR_Mangroves_Mayotte_2017', 'Noh et al. 2020',
       'IUCN_Brazil_Pantanal_2015', 'Uribe et al. 2020', 'HELCOM 1998',
       'Carre_Forest_France_2014', 'Keith et al. 2013',
       'Artsdatabanken 2018', 'Keith et al. 2013', 'Shapiro et al. 2020',
       'Roche_EasterScotianShelf_2020', 'Javed_RLE_AbuDhabi_2021',
       'Carre_RLE_Madagascar_2020', 'Rouget_SouthAfrica_2004',
       'Obura_etal_CoralReef_WIO_2021', 'Mases-Garcia_OaxacaForests_2021',
       'Betancourt_ForestsUruguay_2021', 'IUCN_FR_Forest_France_2018',
       'Jannsen_Europe_2017', 'Essl and Egger 2010', 'SANBI 2009',
       'Biserkov et al. 2011', 'Finck et al. 2009',
       'Ávila-Flores et al. 2020', 'Kontula and Raunio 2009',
       'Chytrý et al. 2019', 'Clark et al. 2015',
       'Payet_

Close database connection


In [201]:
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


## Read data from collection of XML files

Define function to read selected nodes from XML

In [30]:
def extract_IUCN_GET_classification(file):
    records=list()
    tree = ET.parse(file)
    root = tree.getroot()
    for CS in root:
        record={'id':CS.attrib['id'],'name':CS.attrib['name'],
                'ref-label':CS.find('Assessment-Information/ref-label').text}
        for AT in CS.findall('Assessment-Target'):
            for elem in AT.findall("Classifications/Classification-system[@id='IUCN Global Typology']"):
                record['version']=elem.attrib['version']
                record['assigned-by']=elem.attrib['assigned-by']
                for subelem in elem:
                    record['level' + subelem.attrib['level']]=subelem.text
            records.append(record)
    return(records)

Path to the XML collection

In [31]:
xmldb = Path(os.path.expanduser('~')) / 'proyectos' / 'IUCN-RLE' / 'RLE_DB' 

Iterate through folders to read all xml files:

In [32]:
records=list()
for pub_status in ['3.Published','2.Not published' ]:
    for asm_type in ['Strategic','Systematic']:
        xmldir = xmldb / pub_status / asm_type
        for xmlfile in os.listdir(xmldir):
            new_records=extract_IUCN_GET_classification(xmldir / xmlfile)
            records.extend(new_records)
            

In [33]:
print(xmlfile)
len(records)

Pliscoff_RLE_Chile_2015.xml


638

In [34]:
records[10]

{'id': 'Oliveira_TepuiShrublands_2013_1',
 'name': 'Tepui shrublands, Southern Venezuela, LC',
 'ref-label': 'Keith_Foundations_2013',
 'version': 'draft',
 'assigned-by': 'RLE team',
 'level1': 'Terrestrial',
 'level2': 'T3. Shrublands and shrubby woodlands',
 'level3': 'T3.1 Seasonally dry tropical shrublands'}

In [35]:
xml_df=pd.DataFrame(records)


# Compare records from different sources

The list of references is the first challenge, we have used different codes in the GET database and the xml/inventory:

In [184]:
refs_inventory=list(inventory['Reference label'].unique())
print("%s references in the inventory" % len(refs_inventory))
print("%s references in the GET Database" % len(assms_df.ref_code.unique()))
print("%s references extracted from XML files" % len(xml_df['ref-label'].unique()))
res = sum(x == y for x, y in zip(refs_inventory, assms_df.ref_code.to_list()))
print("%s references in common" %res)

69 references in the inventory
81 references in the GET Database
35 references extracted from XML files
0 references in common


In [185]:
assm_inventory=list(inventory['Assessment ID'].unique())
print("%s units in the inventory" % len(assm_inventory))
print("%s references in the GET Database" % len(assms_df.asm_id.unique()))
res = sum(x == y for x, y in zip(assm_inventory, assms_df.asm_id.to_list()))
print("%s references in common" %res)

93 units in the inventory
101 references in the GET Database
0 references in common


In [186]:
common=list()
missing=list()
missing2=list()
for x in assm_inventory:
    if x in assms_df.asm_id.to_list():
        common.append(x)
    else:
        missing.append(x)
        
for x in assms_df.asm_id.to_list():
    if x not in common:
        missing2.append(x)
        
print(len(common))
print(len(missing))
print(len(missing2))
#print(assm_inventory)
#assms_df.asm_id.to_list()

80
13
21


In [187]:
inventory.loc[inventory['Assessment ID'].isin(missing)][['Reference label','Assessment ID']].drop_duplicates()

,Reference label,Assessment ID
40,Sievers_CoastalWetlands_MoretonBay_2020,Sievers_Mangroves_MoretonBay_2020
41,Sievers_CoastalWetlands_MoretonBay_2020,Sievers_Saltmarshes_MoretonBay_2020
42,Sievers_CoastalWetlands_MoretonBay_2020,Sievers_Seagrasses_MoretonBay_2020
43,Ghoraba_BurullusMarineBar_2021,Ghoraba_BurullusSaltMarshes_2021
44,Ghoraba_BurullusMarineBar_2021,Ghoraba_BurullusSandPlain_2021
45,Qiao_S.bungeanaGrassland_China_2020,Qiao_S.bungeanaGrassland_China_2020
49,,Tongatapu mangrove (Pacific Islands)
50,,Mangroves of British Virgin Islands
51,Dellasala_InteriorWetBelt_Canada_2021,Dellasala_ITR_Canada
52,Dellasala_InteriorWetBelt_Canada_2021,Dellasala_IWB_Canada_2021


In [189]:
for k in range(0,12):
    print("%s / %s" % (sorted(missing)[k], sorted(missing2)[k]))
#print(sorted(common))
#sorted(assms_df.asm_id.to_list())
sorted(missing)

Carre_RLE_France_Mangroves_2017 / Artsdatabanken_RLE_Norway_2018
Dellasala_ITR_Canada / Biserkov_Habitats_Bulgaria_2011
Dellasala_IWB_Canada_2021 / Carre_RLE_France_2012
FerrerParis_National_ForestMacrogroup_2019 / CaseStudies_RLE_Venezuela_2010
Ghoraba_BurullusSaltMarshes_2021 / Deltas_at_Risk
Ghoraba_BurullusSandPlain_2021 / Essl_Habitats_Austria_2011
Mangroves of British Virgin Islands / Finck_biotypes_Germany_2011
OliveiraMiranda_RLE_Venezuela_2012 / Ghoraba_Egypt_MarineBar_2021
Qiao_S.bungeanaGrassland_China_2020 / HELCOM_BalticSea_1998
Sievers_Mangroves_MoretonBay_2020 / Huss_Alpine_Cryosphere
Sievers_Saltmarshes_MoretonBay_2020 / IUCN_Brazil_RLE_2015
Sievers_Seagrasses_MoretonBay_2020 / IUCN_FR_Forest_France_2018


['Carre_RLE_France_Mangroves_2017',
 'Dellasala_ITR_Canada',
 'Dellasala_IWB_Canada_2021',
 'FerrerParis_National_ForestMacrogroup_2019',
 'Ghoraba_BurullusSaltMarshes_2021',
 'Ghoraba_BurullusSandPlain_2021',
 'Mangroves of British Virgin Islands',
 'OliveiraMiranda_RLE_Venezuela_2012',
 'Qiao_S.bungeanaGrassland_China_2020',
 'Sievers_Mangroves_MoretonBay_2020',
 'Sievers_Saltmarshes_MoretonBay_2020',
 'Sievers_Seagrasses_MoretonBay_2020',
 'Tongatapu mangrove (Pacific Islands)']

We can compare records from the inventory and the information extracted from the xml files, they should match like this example, where both sources use the same id.

In [132]:
inv_df[inv_df['Ecosystem ID']=='Benson_VineThicket_2013_1']

,Reference label,Ecosystem ID,AT-Name,Realm\nGET_Level 1,Biome\nGET_Level 2,Functional Group\nGET_Level 3,xml table
2,Keith_Foundations_2013,Benson_VineThicket_2013_1,Mock Olive - Wilga - Peach Bush - Carissa dry ...,Terrestrial,T1. Tropical-subtropical forests,T1.2 Tropical-subtropical dry forests and scrubs,True


In [133]:
xml_df[xml_df['id']=='Benson_VineThicket_2013_1']
xml_df[xml_df['id']=='Herrera_RLE_CostaRica_2015_24']


,id,name,AT-id,version,assigned-by,level1,level2,level3,inventory
253,Herrera_RLE_CostaRica_2015_24,10.4. Páramos o vegetación paramera de la Cord...,CRI_10.4_24,Draft,RLE team,Terrestrial,T3. Shrublands and shrubby woodlands,NaN,True


In [167]:
inventory.columns
print("There are {} units in the inventory with information on level 3.".format((inventory['Functional Group\nGET_Level 3']!="").sum()))

There are 367 units in the inventory with information on level 3.


We expect to have many matches between the inventory (inv_df) and the table from the xml collection (xml_df):

In [104]:
xml_df['inventory']=xml_df['id'].isin(inv_df['Ecosystem ID'].unique())
print("There are {} matches out of {} rows".format(xml_df['inventory'].sum(),xml_df.shape[0]))
qry=xml_df[xml_df['inventory']]['level3'].isnull().sum()
print("But {} of the matches do not have the value for level 3".format(qry))

There are 484 matches out of 638 rows
But 143 of the matches do not have the value for level 3


In [168]:
l2col='Biome\nGET_Level 2'
l3col='Functional Group\nGET_Level 3'
ss = xml_df[xml_df['inventory'] & (xml_df['level3'].notnull())]
for index, row in ss.iterrows():
    inv_ss=inventory[inventory['Ecosystem ID']==row['id']][selected_columns]
    if all(inv_ss[l2col] == ""):
        inventory.loc[inventory['Ecosystem ID']==row['id'],l2col]=row['level2']
    if all(inv_ss[l3col] == ""):
        inventory.loc[inventory['Ecosystem ID']==row['id'],l3col]=row['level3']
        #print(inv_ss)
        #print(row['id'], row['level1'], row['level2'], row['level3'])
        #break

In [175]:
(inventory[l3col] != "").sum()

367

We can also do the match the other way around:

In [176]:
inventory['xml table']=inventory['Ecosystem ID'].isin(xml_df['id'].unique())
print("There are {} matches out of {} rows".format(inventory['xml table'].sum(),inv_df.shape[0]))
qry=(inventory[inventory['xml table']]['Functional Group\nGET_Level 3']=="").sum()
if qry>0:
    print("But {} of the matches do not have the value for level 3".format(qry))
else:
    print("All matches have a value for level 3!")

There are 484 matches out of 5290 rows
But 143 of the matches do not have the value for level 3


In [177]:
(inventory['Functional Group\nGET_Level 3']!="").sum()

367

In [195]:
inventory[inventory['xml table']]

,CS Type,Reference label,Ecosystem ID,AT-Name,Year,Language,Assessment Protocol,Scope of Assessment,Realm\nGET_Level 1,Biome\nGET_Level 2,...,DD,NE,Overall_RC,Outcome_RA,Region,Countries,Asessment area (km2),Requiere revisión,Publisher's permission,xml table
0,Strategic,Auld_GnarledMossyCloudForest_2015,Auld_GnarledMossyCloudForest_2015_1,Gnarled Mossy Cloud Forest,2015,English,IUCN RLE 2.0,Sub-global/Sub-national,Terrestrial,T1. Tropical-subtropical forests,...,0,0,CR,,Oceania,AU,,,too expensive,True
1,Strategic,Barrett_MountainSummit_2015,Barrett_MountainSummit_2015_1,Eastern Stirling Range Montane Heath and Thicket,2015,English,IUCN RLE 2.0,Sub-global/Sub-national,Terrestrial,T3. Shrublands and shrubby woodlands,...,0,0,CR,,Oceania,AU,,,too expensive,True
2,Strategic,Keith_Foundations_2013,Benson_VineThicket_2013_1,Mock Olive - Wilga - Peach Bush - Carissa dry ...,2013,English,IUCN RLE 2.0,Sub-global/Sub-national,Terrestrial,T1. Tropical-subtropical forests,...,0,0,EN,,Oceania,AU,,,not needed,True
3,Strategic,Bland_MarineEcosystem_2018,Bland_MarineEcosystem_2018_1,Marine ecosystem of southern Benguela,2018,English,IUCN RLE 2.1,Sub-global/Sub-national,Marine,M1. Marine shelf,...,0,0,EN,,South Africa,ZA,,,already paid,True
4,Strategic,Bland_MesoAmericaReef_2017,Bland_MesoAmericaReef_2017_1,Meso-American Reef,2017,English,IUCN RLE 2.1,Sub-global/Regional,Marine,M1. Marine shelf,...,0,0,CR,,Central America,"MX, BZ, GT, HN",,,not needed,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,Systematic,Carre_RLE_France_CoastalDunes_2020,Carre_RLE_France_CoastalDunes_2020_5,Dunes grises méditerranéennes,2020,French,IUCN RLE 2.1,Sub-global/Sub-national,Marine/Terrestrial,MT2. Supralittoral coastal,...,0,0,VU,,Pending to process,FR,None,None,None,True
1802,Systematic,Carre_RLE_France_CoastalDunes_2020,Carre_RLE_France_CoastalDunes_2020_6,Fruticées dunaires méditerranéennes,2020,French,IUCN RLE 2.1,Sub-global/Sub-national,Marine/Terrestrial,MT2. Supralittoral coastal,...,0,0,LC,,Pending to process,FR,None,None,None,True
1803,Systematic,Carre_RLE_France_CoastalDunes_2020,Carre_RLE_France_CoastalDunes_2020_7,Junipéraies dunaires méditerranéennes,2020,French,IUCN RLE 2.1,Sub-global/Sub-national,Marine/Terrestrial,MT2. Supralittoral coastal,...,0,0,VU,,Pending to process,FR,None,None,None,True
1804,Systematic,Carre_RLE_France_CoastalDunes_2020,Carre_RLE_France_CoastalDunes_2020_8,Dunes boisées méditerranéennes,2020,French,IUCN RLE 2.1,Sub-global/Sub-national,Marine/Terrestrial,MT2. Supralittoral coastal,...,0,0,VU,,Pending to process,FR,None,None,None,True


In [192]:
def changeid(x):
    return x.replace(':','_')

xwalk_df['new_id'] = xwalk_df['eco_id'].apply(changeid)

xwalk_df['inventory']=xwalk_df['new_id'].isin(inventory['Ecosystem ID'].unique())
print("There are {} matches out of {} rows".format(xwalk_df['inventory'].sum(),xwalk_df.shape[0]))


There are 137 matches out of 545 rows


In [193]:
xwalk_df[xwalk_df['inventory']]

,eco_id,eco_name,eco_name_orig,external_code,efg_code,efg_name,level,membership,assigned_by,new_id,inventory
1,Herrera_RLE_CostaRica_2015:24,10.4. Páramos o vegetación paramera de la cord...,10.4. Páramos o vegetación paramera de la cord...,Herrera_RLE_CostaRica_2015:24,T6.5,T6.5 Tropical alpine grasslands and herbfields,6.0,0.10,JR Ferrer-Paris,Herrera_RLE_CostaRica_2015_24,True
2,Herrera_RLE_CostaRica_2015:29,11.5 Páramos de la Cordillera de Talamanca,11.5 Páramos de la Cordillera de Talamanca,Herrera_RLE_CostaRica_2015:29,T6.5,T6.5 Tropical alpine grasslands and herbfields,6.0,0.10,JR Ferrer-Paris,Herrera_RLE_CostaRica_2015_29,True
4,Holdaway_Uncommon_NewZealand_2012:58,Snowbanks,Snowbanks,Holdaway_Uncommon_NewZealand_2012:58,T6.1,"T6.1 Ice sheets, glaciers and perennial snowfi...",6.0,0.01,JR Ferrer-Paris,Holdaway_Uncommon_NewZealand_2012_58,True
5,FerrerParis_Continental_ForestMacrogroup_2019:1,1.A.1.Ea.134 - Caribbean Coastal Lowland Dry F...,1.A.1.Ea.134 - Caribbean Coastal Lowland Dry F...,FerrerParis_Continental_ForestMacrogroup_2019:1,T1.2,T1.2 Tropical/Subtropical dry forests and thic...,5.0,0.10,JR Ferrer Paris,FerrerParis_Continental_ForestMacrogroup_2019_1,True
6,FerrerParis_Continental_ForestMacrogroup_2019:2,1.A.1.Ea.294 - Caribbean Dry Limestone Forest,1.A.1.Ea.294 - Caribbean Dry Limestone Forest,FerrerParis_Continental_ForestMacrogroup_2019:2,T1.2,T1.2 Tropical/Subtropical dry forests and thic...,5.0,0.10,JR Ferrer Paris,FerrerParis_Continental_ForestMacrogroup_2019_2,True
...,...,...,...,...,...,...,...,...,...,...,...
267,Holdaway_Uncommon_NewZealand_2012:19,Cushion bogs,Cushion bogs,Holdaway_Uncommon_NewZealand_2012:19,TF1.6,"TF1.6 Boreal, temperate and montane peat bogs",6.0,0.10,JRFP,Holdaway_Uncommon_NewZealand_2012_19,True
268,Holdaway_Uncommon_NewZealand_2012:47,Recent lava flows,Recent lava flows,Holdaway_Uncommon_NewZealand_2012:47,T3.4,"T3.4 Young rocky pavements, lava flows and screes",6.0,0.10,JRFP,Holdaway_Uncommon_NewZealand_2012_47,True
269,Holdaway_Uncommon_NewZealand_2012:26,Estuary,Estuary,Holdaway_Uncommon_NewZealand_2012:26,FM1.2,FM1.2 Permanently open riverine estuaries and ...,6.0,0.10,JRFP,Holdaway_Uncommon_NewZealand_2012_26,True
299,Etter_RLE_Colombia_2015_44,None,Ha_ZBH-B35 Halobiomas del Zonobioma de Bosque ...,Etter_RLE_Colombia_2015_44,MFT1.2,MFT1.2 Intertidal forests and shrublands,6.0,1.00,Provita,Etter_RLE_Colombia_2015_44,True


In [194]:
ss = xwalk_df[xwalk_df['inventory'] & (xwalk_df['efg_code'].notnull())]
for index, row in ss.iterrows():
    inv_ss=inventory[inventory['Ecosystem ID']==row['new_id']][selected_columns]
#    if all(inv_ss[l2col] == ""):
#        inventory.loc[inventory['Ecosystem ID']==row['id'],l2col]=row['level2']
    if all(inv_ss[l3col] == ""):
#        inventory.loc[inventory['Ecosystem ID']==row['id'],l3col]=row['level3']
        print(inv_ss)
        print(row['new_id'], row['efg_name'])
        break

                Reference label                   Ecosystem ID  \
238  Herrera_RLE_CostaRica_2015  Herrera_RLE_CostaRica_2015_24   

                                               AT-Name Realm\nGET_Level 1  \
238  10.4. Páramos o vegetación paramera de la cord...        Terrestrial   

                       Biome\nGET_Level 2 Functional Group\nGET_Level 3  
238  T3. Shrublands and shrubby woodlands                                
Herrera_RLE_CostaRica_2015_24 T6.5 Tropical alpine grasslands and herbfields


In [183]:
ss

,eco_id,eco_name,eco_name_orig,external_code,efg_code,level,membership,assigned_by,inventory,new_id
1,Herrera_RLE_CostaRica_2015:24,10.4. Páramos o vegetación paramera de la cord...,10.4. Páramos o vegetación paramera de la cord...,Herrera_RLE_CostaRica_2015:24,T6.5,6.0,0.10,JR Ferrer-Paris,True,Herrera_RLE_CostaRica_2015_24
2,Herrera_RLE_CostaRica_2015:29,11.5 Páramos de la Cordillera de Talamanca,11.5 Páramos de la Cordillera de Talamanca,Herrera_RLE_CostaRica_2015:29,T6.5,6.0,0.10,JR Ferrer-Paris,True,Herrera_RLE_CostaRica_2015_29
4,Holdaway_Uncommon_NewZealand_2012:58,Snowbanks,Snowbanks,Holdaway_Uncommon_NewZealand_2012:58,T6.1,6.0,0.01,JR Ferrer-Paris,True,Holdaway_Uncommon_NewZealand_2012_58
5,FerrerParis_Continental_ForestMacrogroup_2019:1,1.A.1.Ea.134 - Caribbean Coastal Lowland Dry F...,1.A.1.Ea.134 - Caribbean Coastal Lowland Dry F...,FerrerParis_Continental_ForestMacrogroup_2019:1,T1.2,5.0,0.10,JR Ferrer Paris,True,FerrerParis_Continental_ForestMacrogroup_2019_1
6,FerrerParis_Continental_ForestMacrogroup_2019:2,1.A.1.Ea.294 - Caribbean Dry Limestone Forest,1.A.1.Ea.294 - Caribbean Dry Limestone Forest,FerrerParis_Continental_ForestMacrogroup_2019:2,T1.2,5.0,0.10,JR Ferrer Paris,True,FerrerParis_Continental_ForestMacrogroup_2019_2
...,...,...,...,...,...,...,...,...,...,...
267,Holdaway_Uncommon_NewZealand_2012:19,Cushion bogs,Cushion bogs,Holdaway_Uncommon_NewZealand_2012:19,TF1.6,6.0,0.10,JRFP,True,Holdaway_Uncommon_NewZealand_2012_19
268,Holdaway_Uncommon_NewZealand_2012:47,Recent lava flows,Recent lava flows,Holdaway_Uncommon_NewZealand_2012:47,T3.4,6.0,0.10,JRFP,True,Holdaway_Uncommon_NewZealand_2012_47
269,Holdaway_Uncommon_NewZealand_2012:26,Estuary,Estuary,Holdaway_Uncommon_NewZealand_2012:26,FM1.2,6.0,0.10,JRFP,True,Holdaway_Uncommon_NewZealand_2012_26
299,Etter_RLE_Colombia_2015_44,None,Ha_ZBH-B35 Halobiomas del Zonobioma de Bosque ...,Etter_RLE_Colombia_2015_44,MFT1.2,6.0,1.00,Provita,True,Etter_RLE_Colombia_2015_44


In [202]:
xml_df.level3.value_counts()
xwalk_df.efg_name.value_counts()
#inventory[l3col].value_counts()

T1.2 Tropical/Subtropical dry forests and thickets              51
T2.2 Deciduous temperate forests                                45
T1.1 Tropical/Subtropical lowland rainforests                   40
T3.4 Young rocky pavements, lava flows and screes               39
TF1.1 Tropical flooded forests and peat forests                 38
T3.2 Seasonally dry temperate heath and shrublands              37
MT1.1 Rocky Shorelines                                          25
MFT1.2 Intertidal forests and shrublands                        24
T1.3 Tropical/Subtropical montane rainforests                   22
T4.4 Temperate woodlands                                        15
TF1.6 Boreal, temperate and montane peat bogs                   14
M1.4 Shellfish beds and reefs                                   12
T4.2 Pyric tussock savannas                                     12
M1.1 Seagrass meadows                                           11
T6.4 Temperate alpine grasslands and shrublands               

In [205]:
xwalk_df[xwalk_df.efg_code=='M1.1']

,eco_id,eco_name,eco_name_orig,external_code,efg_code,efg_name,level,membership,assigned_by,new_id,inventory
242,Bonifacio_CS_SeagrassCommunity_2013:1,Seagrass meadows of South Australia,Seagrass meadows of South Australia,Bonifacio_CS_SeagrassCommunity_2013:1,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,Bonifacio_CS_SeagrassCommunity_2013_1,False
246,Sievers_wetlands_Australia_2020_3,"Coastal wetlands of Moreton Bay, Queensland - ...","Coastal wetlands of Moreton Bay, Queensland - ...",Sievers_wetlands_Australia_2020_3,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,Sievers_wetlands_Australia_2020_3,False
301,Uribe_Marine_Colombia_2020:4,None,Praderas de pastos marinos Caribe oceánico,None,M1.1,M1.1 Seagrass meadows,6.0,1.00,JRFP,Uribe_Marine_Colombia_2020_4,False
302,Uribe_Marine_Colombia_2020:13,None,Praderas de pastos marinos Caribe continental,None,M1.1,M1.1 Seagrass meadows,6.0,1.00,JRFP,Uribe_Marine_Colombia_2020_13,False
491,EU28plus_Habitats:450,A5.5z Seagrass meadows in Pontic moderately ex...,A5.5z Seagrass meadows in Pontic moderately ex...,EU28plus_Habitats:450,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,EU28plus_Habitats_450,False
492,EU28plus_Habitats:449,A5.53 Seagrass and rhizomatous algal meadows i...,A5.53 Seagrass and rhizomatous algal meadows i...,EU28plus_Habitats:449,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,EU28plus_Habitats_449,False
494,EU28plus_Habitats:160,A5.53: Seagrass beds (other than Posidonia) on...,A5.53: Seagrass beds (other than Posidonia) on...,EU28plus_Habitats:160,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,EU28plus_Habitats_160,False
495,EU28plus_Habitats:305,A5.53 Seagrass beds on Atlantic infralittoral ...,A5.53 Seagrass beds on Atlantic infralittoral ...,EU28plus_Habitats:305,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,EU28plus_Habitats_305,False
496,EU28plus_Habitats:431,A5.5w Seagrass meadows in Pontic lower infrali...,A5.5w Seagrass meadows in Pontic lower infrali...,EU28plus_Habitats:431,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,EU28plus_Habitats_431,False
497,EU28plus_Habitats:236,A2.61&nbsp;Seagrass beds on Atlantic littoral ...,A2.61&nbsp;Seagrass beds on Atlantic littoral ...,EU28plus_Habitats:236,M1.1,M1.1 Seagrass meadows,6.0,0.10,JRFP,EU28plus_Habitats_236,False


In [74]:
for i in xml_df['id'].unique():
    if i in inv_df['Ecosystem ID'].unique():
        xml_df[xml_df['id']]['inventory']
    else:
        print('missing')

Kingsford_MurrayDarlingBasin_2013_1
Keith_CaribbeanCoralReefs_2013_1
Bland_MesoAmericaReef_2017_1
Williams_AlpineSnowPatch_2015_1
Riecken_RaisedBogs_2013_1
missing
Clark_IvertebrateComm_2015_1
Wardle_GeorginaGidgee_2015_1
English_SWA_2015_1
Bland_MarineEcosystem_2018_1
missing
missing
Essl_GermanTamarisk_2013_1
Keith_CoastalSandstone_2013_1
Murray_TidalFlats_2015_1
Tozer_CumberlandPlain_2015_1
Sievers_Seagrasses_MoretonBay_2020_1
Keith_GonakierForest_2013_1
Metcalfe_WetTropics_2015_1
missing
Moat_TapiaForest_2013_1
Marshall_FringeMangroves_2018_1
Burns_MountainAshForest_2015_1
Sievers_Saltmarshes_MoretonBay_2020_1
Keith_GiantKelp_2013_1
Keith_CapeFlatsSandFynbos_2013_1
Sievers_SundarbansMangrove_2020_1
Bonifacio_KRSWetland_2013_1
Barrett_MountainSummit_2015_1
missing
Pisanu_LakeEyre_2015_1
Ghoraba_LakeBurullus_2019_1
Sievers_Mangroves_MoretonBay_2020_1
Bonifacio_SeagrassCommunity_2013_1
Keith_CoolibahBlackBox_2013_1
Lester_CoorongLagoons_2013_1
Ghoraba_BurullusSandPlain_2021_1
Mahoney_

In [48]:
df_from_xml['assigned-by'].unique()
df_from_xml['version'].unique()

array(['draft', 'Draft', nan], dtype=object)

In [16]:
list1=df_from_sql['eco_id'].unique()
list2=df_from_xml['id'].unique()

In [19]:
for j in list1:
    if j in list2:
        print(j)

Etter_RLE_Colombia_2015_44
Etter_RLE_Colombia_2015_72


In [34]:
df_from_sql[df_from_sql['eco_id']=='Ghoraba_Egypt_MarineBar_2021:1']

,eco_id,eco_name,eco_name_orig,external_code,efg_code,level,membership,assigned_by
290,Ghoraba_Egypt_MarineBar_2021:1,None,Sand plain ecosystem,None,MT2.1,6.0,1.00,JRFEP


In [35]:
df_from_xml[df_from_xml['id']=='Ghoraba_Egypt_MarineBar_2021_1']

,id,name,AT-id,version,assigned-by,level1,level2,level3


In [33]:
df_from_sql[df_from_sql['eco_id']!=df_from_sql['external_code']]

,eco_id,eco_name,eco_name_orig,external_code,efg_code,level,membership,assigned_by
288,IUCN_FR_Mangroves_Mayotte_2017:2,None,Mangroves mésohalines et mangroves estuariennes,None,MFT1.2,6.0,1.00,JRFEP
289,IUCN_FR_Mangroves_Mayotte_2017:3,None,"Tannes, prés-salés, mangroves et forêts supral...",None,MFT1.2,6.0,1.00,JRFEP
290,Ghoraba_Egypt_MarineBar_2021:1,None,Sand plain ecosystem,None,MT2.1,6.0,1.00,JRFEP
291,Ghoraba_Egypt_MarineBar_2021:2,None,Salt marshes,None,MFT1.3,6.0,1.00,JRFEP
292,Uribe_Marine_Colombia_2020:1,None,Manglares Caribe oceánico,None,MFT1.2,6.0,1.00,JRFEP
293,Uribe_Marine_Colombia_2020:5,None,Manglares Pacífico continental norte,None,MFT1.2,6.0,1.00,JRFEP
294,Uribe_Marine_Colombia_2020:9,None,Manglares Pacífico continental sur,None,MFT1.2,6.0,1.00,JRFEP
295,Uribe_Marine_Colombia_2020:10,None,Manglares Caribe continental,None,MFT1.2,6.0,1.00,JRFEP
296,CaseStudies_RLE_Venezuela_2010:1,None,Laguna de Tacarigua,None,MFT1.2,6.0,1.00,JRFEP
297,CaseStudies_RLE_Venezuela_2010:2,None,Ciénaga de Los Olivitos,None,MFT1.2,6.0,1.00,JRFEP


In [206]:
inventory['Reference label'].value_counts()

Comer_ForestsAmerica_2020                     749
FerrerParis_National_ForestMacrogroup_2019    577
Skowno_SouthAfrica_2019                       458
Rouget_SouthAfrica_2004                       438
Kontula_FinlandHabitat_2018                   388
                                             ... 
Qiao_S.bungeanaGrassland_China_2020             1
Schaefer_RockyShores_NSW_2020                   1
Uribe_Colombia_Corals_2021                      1
Roche_EasterScotianShelf_2020                   1
Auld_GnarledMossyCloudForest_2015               1
Name: Reference label, Length: 69, dtype: int64

In [193]:
assms_df[assms_df.ref_code=='Keith et al. 2013']

,asm_id,ref_code
6,Keith_CoolibahBlackBox_2013,Keith et al. 2013
15,Keith_GiantKelp_2013,Keith et al. 2013
17,Bonifacio_KRSWetland_2013,Keith et al. 2013
46,Riecken_RaisedBogs_2013,Keith et al. 2013
54,Kingsford_MurrayDarlingBasin_2013,Keith et al. 2013
55,Essl_GermanTamarisk_2013,Keith et al. 2013
56,Keith_AralSea_2013,Keith et al. 2013
58,OliveiraMiranda_TepuiShrublands_2013,Keith et al. 2013
62,Bonifacio_SeagrassCommunity_2013,Keith et al. 2013
65,Keith_CaribbeanCoralReefs_2013,Keith et al. 2013
